In [1]:
import cv2
from cvzone.HandTrackingModule import HandDetector
import numpy as np
import math
import time
import os
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [16]:
train_dir = r'C:\Users\Hp\Documents\capstone\Data\Train\images'
validation_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation\images'
test_dir = r'C:\Users\Hp\Documents\capstone\Data\Test\images'

In [17]:
# Function to load images from a folder
def load_images_from_folder(folder, target_size=(100, 100)):
    images = []
    labels = []
    for filename in os.listdir(folder):
        try:
            img = cv2.imread(os.path.join(folder, filename))
            if img is not None:
                img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                img = cv2.resize(img, target_size)
                img = img / 255.0  # Normalize pixel values to [0, 1]
                images.append(img)

                # Extract the class name from the filename (e.g., "A" from "A_image.jpg")
                class_name = filename[0]  # Assumes that class names are the first character in the filename
                # Map the class name to the integer label using the class_to_label mapping
                label = ord(class_name) - ord('A')  # Convert 'A' to 0, 'B' to 1, and so on
                labels.append(label)
        except Exception as e:
            print(f"Error loading file: {filename} - {str(e)}")
    return np.array(images), np.array(labels)

In [ ]:
for folder in os.listdir(train_dir):
    load_images_from_folder(folder, target_size= (100,100)
                            
                            

In [18]:
# Function to load video frames from a folder
def load_video_frames_from_folder(folder, num_frames_per_video = 10, target_size=(300, 300)):
    video_frames = []
    labels = []
    for filename in os.listdir(folder):
        try:
            video_path = os.path.join(folder, filename)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_count % (cap.get(cv2.CAP_PROP_FRAME_COUNT) // num_frames_per_video) == 0:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
                    frame = cv2.resize(frame, target_size)
                    frame = frame / 255.0  # Normalize pixel values to [0, 1]
                    video_frames.append(frame)

                    # Extract the class name from the filename (e.g., "A" from "A_video.mp4")
                    class_name = filename[0]  # Assumes that class names are the first character in the filename
                    # Map the class name to the integer label using the class_to_label mapping
                    label = ord(class_name) - ord('A')  # Convert 'A' to 0, 'B' to 1, and so on
                    labels.append(label)
                frame_count += 1
                if len(video_frames) >= num_frames_per_video:
                    break
        except Exception as e:
            print(f"Error loading file: {filename} - {str(e)}")
    return np.array(video_frames), np.array(labels)

In [ ]:
train_images_dir  

In [19]:
# Load images and labels from image folders
try:
    print("Loading train images...")
    train_images, train_labels_images = load_images_from_folder(train_dir)
    print(f"Loaded {len(train_images)} train images.")
    print("Loading test images...")
    test_images, test_labels_images = load_images_from_folder(test_dir)
    print(f"Loaded {len(test_images)} test images.")
    print("Loading validation images...")
    validation_images, validation_labels_images = load_images_from_folder(validation_dir)
    print(f"Loaded {len(validation_images)} validation images.")
except Exception as e:
    print(f"Error loading image data: {str(e)}")

Loading train images...
Loaded 0 train images.
Loading test images...
Loaded 0 test images.
Loading validation images...
Loaded 0 validation images.


In [ ]:
# Load video frames and labels from video folders
try:
    num_frames_per_video = 10  # Define num_frames_per_video here
    print("Loading train video frames...")
    train_video_frames, train_labels = load_video_frames_from_folder(train_videos_dir, num_frames_per_video)
    print(f"Loaded {len(train_video_frames)} train video frames.")
    print("Loading test video frames...")
    test_video_frames, test_labels = load_video_frames_from_folder(test_videos_dir, num_frames_per_video)
    print(f"Loaded {len(test_video_frames)} test video frames.")
    print("Loading validation video frames...")
    validation_video_frames, validation_labels = load_video_frames_from_folder(validation_videos_dir, num_frames_per_video)
    print(f"Loaded {len(validation_video_frames)} validation video frames.")
except Exception as e:
    print(f"Error loading video data: {str(e)}")

In [ ]:
# Paths to your Train, Test, and Validation folders for images and videos
train_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Train\images'
test_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Test\images'
validation_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation\images'

train_videos_dir =  r'C:\Users\Hp\Documents\capstone\Data\Train\videos'
test_videos_dir = r'C:\Users\Hp\Documents\capstone\Data\Test\videos'
validation_videos_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation\videos'

In [ ]:
# Paths to your Train, Test, and Validation folders for images and videos
train_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Train\images'
test_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Test\images'
validation_images_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation\images'

train_videos_dir =  r'C:\Users\Hp\Documents\capstone\Data\Train\videos'
test_videos_dir = r'C:\Users\Hp\Documents\capstone\Data\Test\videos'
validation_videos_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation\videos'

In [ ]:
# Combine image and video labels
train_labels = np.concatenate((train_labels_images, train_labels))
test_labels = np.concatenate((test_labels_images, test_labels))
validation_labels = np.concatenate((validation_labels_images, validation_labels))

In [ ]:
# Define the image model architecture
model_images = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(300, 300, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(26, activation='softmax')  # 26 classes for 'A' to 'Z'
])

In [ ]:
# Compile the image model
model_images.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the image model
model_images.fit(train_images, train_labels, epochs=10, validation_data=(validation_images, validation_labels))

# Evaluate the image model on the test set
test_loss_images, test_acc_images = model_images.evaluate(test_images, test_labels)
print("Image Test accuracy:", test_acc_images)

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Function to load images from a folder
def load_images_from_folder(folder, target_size=(32, 32)):
    images = []
    labels = []
    for class_name in os.listdir(folder):
        class_folder = os.path.join(folder, class_name)
        if os.path.isdir(class_folder):
            for filename in os.listdir(class_folder):
                try:
                    img = cv2.imread(os.path.join(class_folder, filename))
                    if img is not None:
                        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
                        img = cv2.resize(img, target_size)
                        img = img / 255.0
                        images.append(img)

                        # Assign class label based on the subfolder (class folder) name
                        label = ord(class_name) - ord('A')
                        labels.append(label)
                except Exception as e:
                    print(f"Error loading file: {filename} - {str(e)}")
    return np.array(images), np.array(labels)

# Function to load video frames from a folder
def load_video_frames_from_folder(folder, num_frames_per_video=10, target_size=(32, 32)):
    video_frames = []
    labels = []
    for filename in os.listdir(folder):
        try:
            video_path = os.path.join(folder, filename)
            cap = cv2.VideoCapture(video_path)
            frame_count = 0
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                if frame_count % (cap.get(cv2.CAP_PROP_FRAME_COUNT) // num_frames_per_video) == 0:
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                    frame = cv2.resize(frame, target_size)
                    frame = frame / 255.0
                    video_frames.append(frame)

                    # Extract the class name from the filename (e.g., "A" from "A_video.mp4")
                    class_name = filename[0]  # Assumes that class names are the first character in the filename
                    # Map the class name to the integer label using the class_to_label mapping
                    label = ord(class_name) - ord('A')  # Convert 'A' to 0, 'B' to 1, and so on
                    labels.append(label)
                frame_count += 1
                if len(video_frames) >= num_frames_per_video:
                    break
        except Exception as e:
            print(f"Error loading file: {filename} - {str(e)}")
    return np.array(video_frames), np.array(labels)

# Paths to your Train, Test, and Validation folders for images and videos
train_dir = r'C:\Users\Hp\Documents\capstone\Data\Train'
test_dir = r'C:\Users\Hp\Documents\capstone\Data\Test'
validation_dir = r'C:\Users\Hp\Documents\capstone\Data\Validation'

train_images_dir = os.path.join(train_dir, 'images')
test_images_dir = os.path.join(test_dir, 'images')
validation_images_dir = os.path.join(validation_dir, 'images')

train_videos_dir = os.path.join(train_dir, 'videos')
test_videos_dir = os.path.join(test_dir, 'videos')
validation_videos_dir = os.path.join(validation_dir, 'videos')

# Load images and labels from image subfolders
try:
    print("Loading train images...")
    train_images, train_labels_images = load_images_from_folder(train_images_dir)
    print(f"Loaded {len(train_images)} train images.")
    print("Loading test images...")
    test_images, test_labels_images = load_images_from_folder(test_images_dir)
    print(f"Loaded {len(test_images)} test images.")
    print("Loading validation images...")
    validation_images, validation_labels_images = load_images_from_folder(validation_images_dir)
    print(f"Loaded {len(validation_images)} validation images.")
except Exception as e:
    print(f"Error loading image data: {str(e)}")

# Load video frames and labels from video folders
try:
    num_frames_per_video = 10  # Define num_frames_per_video here
    print("Loading train video frames...")
    train_video_frames, train_labels = load_video_frames_from_folder(train_videos_dir, num_frames_per_video)
    print(f"Loaded {len(train_video_frames)} train video frames.")
    print("Loading test video frames...")
    test_video_frames, test_labels = load_video_frames_from_folder(test_videos_dir, num_frames_per_video)
    print(f"Loaded {len(test_video_frames)} test video frames.")
    print("Loading validation video frames...")
    validation_video_frames, validation_labels = load_video_frames_from_folder(validation_videos_dir, num_frames_per_video)
    print(f"Loaded {len(validation_video_frames)} validation video frames.")
except Exception as e:
    print(f"Error loading video data: {str(e)}")

# Combine image and video labels
train_labels = np.concatenate((train_labels_images, train_labels))
test_labels = np.concatenate((test_labels_images, test_labels))
validation_labels = np.concatenate((validation_labels_images, validation_labels))

# Define the image model architecture
model_images = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(26, activation='softmax')  # 26 classes for 'A' to 'Z'
])

# Compile the image model
model_images.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the image model
model_images.fit(train_images, train_labels, epochs=10, validation_data=(validation_images, validation_labels))

# Evaluate the image model on the test set
test_loss_images, test_acc_images = model_images.evaluate(test_images, test_labels)
print("Image Test accuracy:", test_acc_images)

# Define a separate model for video data (customize the architecture as needed)
model_video = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(26, activation='softmax')  # 26 classes for 'A' to 'Z'
])

# Compile the video model
model_video.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the video model
model_video.fit(train_video_frames, train_labels, epochs=10, validation_data=(validation_video_frames, validation_labels))

# Evaluate the video model on the test set
test_loss_video, test_acc_video = model_video.evaluate(test_video_frames, test_labels)
print("Video Test accuracy:", test_acc_video)
